In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from category_encoders import OneHotEncoder, TargetEncoder
from scipy import stats

# Cargar los conjuntos de datos
ruta_train = 'C:/Users/Marcio Pineda/Documents/Archivos Python/datasets/traincase.csv'
ruta_test = 'C:/Users/Marcio Pineda/Documents/Archivos Python/datasets/testcase.csv'
df_train = pd.read_csv(ruta_train)
df_test = pd.read_csv(ruta_test)

# Convertir 'Keyword' a minúsculas
df_train['Keyword'] = df_train['Keyword'].str.lower()
df_test['Keyword'] = df_test['Keyword'].str.lower()

# Función para limpiar columnas numéricas
def clean_numeric_column(column):
    column_as_str = column.astype(str).str.replace(',', '').str.replace('$', '').str.strip()
    return pd.to_numeric(column_as_str, errors='coerce')

# Limpiar las columnas numéricas
columns_to_clean = ['Search Engine Bid', 'Impressions', 'Avg. Cost per Click', 'Avg. Pos.', 'Clicks']
for column in columns_to_clean:
    df_train[column] = clean_numeric_column(df_train[column])
    if column != 'Clicks':  # 'Clicks' no está en df_test
        df_test[column] = clean_numeric_column(df_test[column])

# Reemplazar ceros en 'Impressions' para la transformación Box-Cox
min_value = df_train.loc[df_train['Impressions'] > 0, 'Impressions'].min()
df_train['Impressions'] = df_train['Impressions'].replace(0, min_value)
fitted_lambda, _ = stats.boxcox(df_train.loc[df_train['Impressions'] > 0, 'Impressions'])
df_train['Impressions'] = stats.boxcox(df_train['Impressions'], lmbda=fitted_lambda)

# Visualizar los primeros registros del conjunto de entrenamiento
df_train.head()



# Después de haber convertido las matrices TF-IDF a DataFrames y concatenado con los conjuntos de datos originales

print(f"Con max_features={max_features}, hay {features_all_zero} características completamente ceros.")

# Al final, tienes un resumen de cuántas características completamente ceros hay para cada valor de max_features
print("Resumen de características completamente ceros por configuración de max_features:", features_all_zero_counts)

Con max_features=600, hay 2 características completamente ceros.
Resumen de características completamente ceros por configuración de max_features: [(600, 2)]


In [43]:
# Asegúrate de tener los datos cargados y las columnas numéricas ya limpias y transformadas

columns_to_drop = ['Clicks', 'entry_id']  # 'Keyword' solo se incluye si aún no se ha procesado

# Eliminar las columnas que ya no se necesitan o que ya han sido procesadas
if 'Keyword' not in df_train.columns:
    columns_to_drop.append('Keyword')
X = df_train.drop(columns=columns_to_drop, errors='ignore')
y = df_train['Clicks']

# Asegúrate de que todas las columnas que se van a procesar con One-Hot Encoding o Target Encoding existen
one_hot_cols = [col for col in ['Match Type', 'Bid Strategy', 'Status'] if col in X.columns]
target_encode_cols = [col for col in ['Campaign', 'Category', 'Publisher Name', 'Keyword Group'] if col in X.columns]

# Aplicar One-Hot Encoding a variables con pocas categorías
one_hot_cols = ['Match Type', 'Bid Strategy', 'Status']
one_hot_encoder = OneHotEncoder(cols=one_hot_cols, use_cat_names=True)
X_train_one_hot = one_hot_encoder.fit_transform(X_train[one_hot_cols])
X_val_one_hot = one_hot_encoder.transform(X_val[one_hot_cols])

# Aplicar Target Encoding a variables con un número moderado de categorías
target_encode_cols = ['Campaign', 'Category', 'Publisher Name', 'Keyword Group']
target_encoder = TargetEncoder(cols=target_encode_cols)
X_train_target_encoded = target_encoder.fit_transform(X_train[target_encode_cols], y_train)
X_val_target_encoded = target_encoder.transform(X_val[target_encode_cols])

# Unir las transformaciones con el resto de datos (excluyendo las columnas originales que ya han sido codificadas)
X_train_processed = X_train.drop(one_hot_cols + target_encode_cols, axis=1).join(X_train_one_hot).join(X_train_target_encoded)
X_val_processed = X_val.drop(one_hot_cols + target_encode_cols, axis=1).join(X_val_one_hot).join(X_val_target_encoded)

# Ahora X_train_processed y X_val_processed están listos para ser usados en el modelado

print("Dimensiones de X_train procesado:", X_train_processed.shape)
print("Dimensiones de X_val procesado:", X_val_processed.shape)

print("Primeras filas de X_train procesado:\n", X_train_processed.head())
print("Primeras filas de X_val procesado:\n", X_val_processed.head())

print("Resumen estadístico de X_train procesado:\n", X_train_processed.describe())



Dimensiones de X_train procesado: (3528, 685)
Dimensiones de X_val procesado: (882, 685)
Primeras filas de X_train procesado:
       Search Engine Bid  Impressions  Avg. Pos.  Avg. Cost per Click  2006  \
2588              10.00     4.606787       1.00                 0.28   0.0   
2354               0.78     6.413341       2.45                 1.08   0.0   
655                6.25     4.202726       1.05                 1.29   0.0   
4339               0.13     6.155182       3.63                 0.35   0.0   
3557               8.75     5.349669       1.14                 1.25   0.0   

      abidjan  abou  aeroport  affaires  agadir  ...  Bid Strategy_nan  \
2588      0.0   0.0       0.0       0.0     0.0  ...                 1   
2354      0.0   0.0       0.0       0.0     0.0  ...                 0   
655       0.0   0.0       0.0       0.0     0.0  ...                 1   
4339      0.0   0.0       0.0       0.0     0.0  ...                 0   
3557      0.0   0.0       0.0     

In [41]:
# Para One-Hot Encoding
for col in one_hot_cols:
    print(f"Valores únicos en {col} después de One-Hot Encoding:", len(X_train_processed.filter(like=col).columns))

# Para Target Encoding
for col in target_encode_cols:
    print(f"Valores únicos en {col} después de Target Encoding en entrenamiento:", X_train_processed[col].nunique())
    print(f"Valores únicos en {col} después de Target Encoding en validación:", X_val_processed[col].nunique())


Valores únicos en Match Type después de One-Hot Encoding: 5
Valores únicos en Bid Strategy después de One-Hot Encoding: 9
Valores únicos en Status después de One-Hot Encoding: 5
Valores únicos en Campaign después de Target Encoding en entrenamiento: 24
Valores únicos en Campaign después de Target Encoding en validación: 23
Valores únicos en Category después de Target Encoding en entrenamiento: 93
Valores únicos en Category después de Target Encoding en validación: 72
Valores únicos en Publisher Name después de Target Encoding en entrenamiento: 7
Valores únicos en Publisher Name después de Target Encoding en validación: 7
Valores únicos en Keyword Group después de Target Encoding en entrenamiento: 187
Valores únicos en Keyword Group después de Target Encoding en validación: 148


In [48]:
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder as SklearnOneHotEncoder
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import category_encoders as ce
from sklearn.pipeline import Pipeline

# Identificar columnas para cada tipo de encoding
one_hot_cols = ['Match Type', 'Bid Strategy', 'Status']
target_encode_cols = ['Campaign', 'Category', 'Publisher Name', 'Keyword Group']

# Separar las características y la variable objetivo
X = df_train.drop(columns=['Clicks', 'entry_id', 'Keyword'], errors='ignore')
y = df_train['Clicks']

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Preparar el ColumnTransformer con OneHotEncoder y TargetEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('one_hot', SklearnOneHotEncoder(), one_hot_cols),
        ('target', ce.TargetEncoder(), target_encode_cols)
    ],
    remainder='passthrough'
)

# Crear el pipeline con el preprocesador y el modelo
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', Ridge())
])

# Entrenar el pipeline con los datos de entrenamiento
model_pipeline.fit(X_train, y_train)

# Evaluar el pipeline con los datos de validación
y_pred_val = model_pipeline.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"RMSE en validación: {rmse_val}")

ValueError: Length mismatch: Expected axis has 1320 elements, new values have 662 elements